In [2]:
import time as time
import datetime as datetime
import pandas as pd
import yfinance as yf
import matplotlib as plt
from backtesting import Strategy, Backtest
from backtesting.lib import crossover

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
#Ticker in url
ticker = 'TSLA'

#Timeperiods of data set "Y/M/D/time"
#this will help when selecting the desired dates and will pull the data set from yahoo finance.
period1 = int(time.mktime(datetime.datetime(2018, 12, 1, 23, 59).timetuple()))
period2 = int(time.mktime(datetime.datetime(2022, 12, 31, 23, 59).timetuple()))
interval = '1d' # 1wk, 1m

#Yahoo Finance url
url = f'https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={period1}&period2={period2}&interval={interval}&events=history&includeAdjustedClose=true'


In [3]:
pip install ta-lib

  Using cached TA-Lib-0.4.25.tar.gz (271 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for ta-lib: filename=TA_Lib-0.4.25-cp310-cp310-macosx_10_9_universal2.whl size=476149 sha256=13927315c8b81425a8114a9b45a135500569eff2e5e0443e03d265c1af5fe42f
  Stored in directory: /Users/carsonpalmer/Library/Caches/pip/wheels/ac/12/d0/2ea05cfb5b9c4bbf47b45dd93ff5baed86ddb85c748166cb6f
Successfully built ta-lib
Note: you may need to restart the kernel to use updated packages.


In [6]:
#shaping data frame to backtesting requirements
df = pd.read_csv(url)
columns = ['Date', 'Open', 'High', 'Low', 'Close', 'adj close', 'Volume']
df.columns = columns
df

,Date,Open,High,Low,Close,adj close,Volume
0,2018-12-03,24.000000,24.400000,23.466667,23.899332,23.899332,124597500
1,2018-12-04,23.736668,24.578667,23.466667,23.980000,23.980000,126928500
2,2018-12-06,23.733999,24.492001,23.384001,24.204000,24.204000,117637500
3,2018-12-07,24.600000,25.299334,23.843332,23.864668,23.864668,172668000
4,2018-12-10,24.000000,24.398666,23.541332,24.343332,24.343332,99202500
...,...,...,...,...,...,...,...
1022,2022-12-23,126.370003,128.619995,121.019997,123.150002,123.150002,166989700
1023,2022-12-27,117.500000,119.669998,108.760002,109.099998,109.099998,208643400
1024,2022-12-28,110.349998,116.269997,108.239998,112.709999,112.709999,221070500
1025,2022-12-29,120.389999,123.570000,117.500000,121.820000,121.820000,221923300


In [7]:
df = df.set_index(pd.DatetimeIndex(df['Date'].values))
df

,Date,Open,High,Low,Close,adj close,Volume
2018-12-03,2018-12-03,24.000000,24.400000,23.466667,23.899332,23.899332,124597500
2018-12-04,2018-12-04,23.736668,24.578667,23.466667,23.980000,23.980000,126928500
2018-12-06,2018-12-06,23.733999,24.492001,23.384001,24.204000,24.204000,117637500
2018-12-07,2018-12-07,24.600000,25.299334,23.843332,23.864668,23.864668,172668000
2018-12-10,2018-12-10,24.000000,24.398666,23.541332,24.343332,24.343332,99202500
...,...,...,...,...,...,...,...
2022-12-23,2022-12-23,126.370003,128.619995,121.019997,123.150002,123.150002,166989700
2022-12-27,2022-12-27,117.500000,119.669998,108.760002,109.099998,109.099998,208643400
2022-12-28,2022-12-28,110.349998,116.269997,108.239998,112.709999,112.709999,221070500
2022-12-29,2022-12-29,120.389999,123.570000,117.500000,121.820000,121.820000,221923300


In [8]:
df.drop('Date', inplace=True, axis=1)
df.drop('adj close', inplace=True, axis=1)
df

,Open,High,Low,Close,Volume
2018-12-03,24.000000,24.400000,23.466667,23.899332,124597500
2018-12-04,23.736668,24.578667,23.466667,23.980000,126928500
2018-12-06,23.733999,24.492001,23.384001,24.204000,117637500
2018-12-07,24.600000,25.299334,23.843332,23.864668,172668000
2018-12-10,24.000000,24.398666,23.541332,24.343332,99202500
...,...,...,...,...,...
2022-12-23,126.370003,128.619995,121.019997,123.150002,166989700
2022-12-27,117.500000,119.669998,108.760002,109.099998,208643400
2022-12-28,110.349998,116.269997,108.239998,112.709999,221070500
2022-12-29,120.389999,123.570000,117.500000,121.820000,221923300


In [163]:
#OSCILLATOR FORMULA
def OSC():
    #14 day low average 
    L14 = df['Low'].rolling(14).min()
    H14 = df['High'].rolling(14).max()
    
#formula variables 
    Clo = df['Close']
    Kval = ((C - L14) * 100 / (H14 - L14))

    Dval = K.rolling(3).mean()   

In [9]:
#Oscillator backtesting strategy

import talib

class Oscillator(Strategy): 
    
    
    def init(self):
       
    
    def next(self):
        
        
        if :
            self.position.close
            self.sell()
        
        elif :
            self.position.close
            self.buy()
           
     

In [187]:
bt = Backtest(df, Oscillator, cash=10_000)
stats = bt.run()
print(stats)
#watch the video
#ttps://www.youtube.com/watch?v=kz_NJERCgm8

Start                     2018-12-03 00:00:00
End                       2022-12-30 00:00:00
Duration                   1488 days 00:00:00
Exposure Time [%]                   23.466407
Equity Final [$]                          0.0
Equity Peak [$]                  20136.450407
Return [%]                             -100.0
Buy & Hold Return [%]              415.411895
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]              436.866111
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                  -13.243216
Max. Drawdown Duration     1306 days 00:00:00
Avg. Drawdown Duration      113 days 00:00:00
# Trades                                   15
Win Rate [%]                              0.0
Best Trade [%]                      -0.985932
Worst Trade [%]                    -93.800878
Avg. Trade [%]                    

##Bollinger Bands

In [188]:
#BB formula and variables
ma_30 = df['Close'].rolling(window=30).mean()
std_20 = df['Close'].rolling(window=20).std() 
upper = (ma_30 + 2*std_20)
lower = (ma_30 - 2*std_20)

bsell = df['Close'].loc[df['Close']>upper]
bbuy = df['Close'].loc[df['Close']<lower]

In [190]:
class Bollinger(Strategy): 
    
    #define the Oscillator strategy with the variables etc
    #self is used for the backtesting.py thing. Idk why but it works
    def init(self):
        df
        self.upper = (ma_30 + 2*std_20)
        self.lower = (ma_30 - 2*std_20)
    
    def next(self):
        
        #define when to sell and buy
        #.any() gives a numerical value instead of a boolean value
        #so that it is not ambiguous
        if df['Close'].loc[df['Close']>upper].any():
            self.position.close
            self.sell()
        
        elif df['Close'].loc[df['Close']<lower].any():
            self.position.close
            self.buy()

In [191]:
bt = Backtest(df, Bollinger, cash=3_000)
stats = bt.run()
print(stats)

Start                     2018-12-03 00:00:00
End                       2022-12-30 00:00:00
Duration                   1488 days 00:00:00
Exposure Time [%]                   23.466407
Equity Final [$]                          0.0
Equity Peak [$]                   6032.506589
Return [%]                             -100.0
Buy & Hold Return [%]              415.411895
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]              436.662424
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                      -100.0
Avg. Drawdown [%]                  -13.232683
Max. Drawdown Duration     1306 days 00:00:00
Avg. Drawdown Duration      113 days 00:00:00
# Trades                                   15
Win Rate [%]                              0.0
Best Trade [%]                      -0.985932
Worst Trade [%]                    -93.800878
Avg. Trade [%]                    

In [195]:
plt(stats)

TypeError: 'module' object is not callable